<a href="https://colab.research.google.com/github/gargarchit/Federated-Learning-with-Encrypted-Gradient-Aggregation/blob/master/Federated_Learning_with_Encrypted_Gradient_Aggregation_using_PyTorch_%2B_PySyft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Need to train on the MNIST dataset using federated learning However the gradient should not come up to central server in raw form**

In [0]:
import torch
from torchvision import datasets, transforms

<h2>Installing and Importing PySyft

In [2]:
!pip install tf-encrypted

! URL="https://github.com/openmined/PySyft.git" && FOLDER="PySyft" && if [ ! -d $FOLDER ]; then git clone -b dev --single-branch $URL; else (cd $FOLDER && git pull $URL && cd ..); fi;

!cd PySyft; python setup.py install  > /dev/null

import os
import sys
module_path = os.path.abspath(os.path.join('./PySyft'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
!pip install --upgrade --force-reinstall lz4
!pip install --upgrade --force-reinstall websocket
!pip install --upgrade --force-reinstall websockets
!pip install --upgrade --force-reinstall zstd

     |████████████████████████████████| 1.4MB 3.6MB/s 
     |████████████████████████████████| 276kB 62.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Cloning into 'PySyft'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 28035 (delta 0), reused 1 (delta 0), pack-reused 28034
Receiving objects: 100% (28035/28035), 31.83 MiB | 26.94 MiB/s, done.
Resolving deltas: 100% (18523/18523), done.
zip_safe flag not set; analyzing archive contents...
zip_safe flag not set; analyzing archive contents...
__pycache__.zstd.cpython-36: module references __file__
     |████████████████████████████████| 389kB 3.5MB/s 
ERROR: syft 0.1.21a1 has requirement msgpack>=0.6.1, but you'll have msgpack 0.5.6 which is incompatible.
  Found existing installati

In [6]:
import syft as sy
hook = sy.TorchHook(torch)

# Create a couple of workers
bob = sy.VirtualWorker(hook, id="bob")  
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")
# clearing data of virutal workers.
bob.clear_objects
alice.clear_objects
secure_worker.clear_objects
compute_nodes = [bob, alice]

W0705 11:46:37.567617 140710606256000 hook.py:98] Torch was already hooked... skipping hooking process


In [7]:
#We define the setting of the learning task

class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

torch.manual_seed(args.seed)

In [8]:
use_cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


<h2>Data loading and sending to workers</h2>

In [9]:
# FederatedDataLoader 
# we distribute the dataset across all the workers, it's now a FederatedDataset
transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))]) 
mnist_trainset = datasets.MNIST(root='../data', train=True, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=args.test_batch_size, shuffle=True)


mnist_testset = datasets.MNIST(root='../data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=args.test_batch_size, shuffle=True)

private_test_loader = []


  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 20864201.10it/s]                            


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 311986.87it/s]                           
0it [00:00, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 5348977.32it/s]                           
8192it [00:00, 131474.23it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


<h2>Network Architecture</h2>

In [0]:
from torch import nn, optim
import torch.nn.functional as F
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [0]:
class Classifier(nn.Module):
    """
    Forward Convolutional Neural Network Architecture model
    
    """
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [0]:
model = Classifier()
model = model.to(device)

In [0]:
bob_optimizer = optim.SGD(bob_model.parameters(), lr=args.lr)
alice_optimizer = optim.SGD(alice_model.parameters(), lr=args.lr)

In [13]:
for epoch in range(1, args.epochs + 1):
    
    
    bob_model.train()
    for batch_idx, (data, target) in enumerate(bob_train_loader): # iterate through each worker's dataset
        
        #bob_model.send(bob) #send the model to the right location
        
        data, target = data.to(device), target.to(device)
        
        bob_optimizer.zero_grad() # 1) erase previous gradients (if they exist)
        output = bob_model(data)  # 2) make a prediction
        loss = F.nll_loss(output, target)  # 3) calculate how much we missed
        loss.backward()  # 4) figure out which weights caused us to miss
        bob_optimizer.step()  # 5) change those weights
        bob_model.get()  # get the model back (with gradients)
        
        if batch_idx % args.log_interval == 0:
            loss = loss.get() #get the loss back
            print('Epoch: {} [Training: {:.0f}%]\tLoss: {:.6f}'.format(epoch, 100. * batch_idx / len(bob_train_loader), loss.item()))
    bob_model.eval()
    
    
    alice_model.train()
    for batch_idx, (data, target) in enumerate(alice_train_loader): # iterate through each worker's dataset
        
        #alice_model.send(alice) #send the model to the right location
        
        data, target = data.to(device), target.to(device)
        
        alice_optimizer.zero_grad() # 1) erase previous gradients (if they exist)
        output = alice_model(data)  # 2) make a prediction
        loss = F.nll_loss(output, target)  # 3) calculate how much we missed
        loss.backward()  # 4) figure out which weights caused us to miss
        alice_optimizer.step()  # 5) change those weights
        alice_model.get()  # get the model back (with gradients)
        
        if batch_idx % args.log_interval == 0:
            loss = loss.get() #get the loss back
            print('Epoch: {} [Training: {:.0f}%]\tLoss: {:.6f}'.format(epoch, 100. * batch_idx / len(alice_train_loader), loss.item()))
    alice_model.eval()
    
    
#     test_loss = 0
#     correct = 0
#     with torch.no_grad():
#         for data, target in test_loader:
            
#             data, target = data.to(device), target.to(device)
#             output = model(data)
            0.
#             test_loss += F.nll_loss(output, target, reduction='sum').item()
#             pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            
#             correct += pred.eq(target.view_as(pred)).sum().item()

#     test_loss /= len(test_loader.dataset)

#     print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))
    
if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

RuntimeError: ignored